In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
url = '/content/drive/MyDrive/AlmaBetter /Capstone Project/Book Recommendation System/Books_data.csv' 
books_url = '/content/drive/MyDrive/AlmaBetter /Capstone Project/Book Recommendation System/Book RCD Dataset /Copy of Books.csv' 

In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv(url , dtype = {'ISBN': str, 'Rating': int, 'Uid': str} )
books = pd.read_csv(books_url)

<ipython-input-4-b2330a23dfaf>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(books_url)


In [ ]:
df.head()

,Unnamed: 0,Uid,ISBN,Rating,Title,Author,Pub_Year,Publisher,Image_Url,Location,Age
0,3,278418,0446520802,0,the notebook,nicholas sparks,1996,warner books,http://images.amazon.com/images/p/0446520802.0...,"omaha, nebraska, usa",37.394772
1,4,278418,0671537458,0,waiting to exhale,terry mcmillan,1995,pocket,http://images.amazon.com/images/p/0671537458.0...,"omaha, nebraska, usa",37.394772
2,5,278418,0786817070,0,"artemis fowl (artemis fowl, book 1)",eoin colfer,2002,miramax kids,http://images.amazon.com/images/p/0786817070.0...,"omaha, nebraska, usa",37.394772
3,6,278418,0446364193,0,along came a spider (alex cross novels),james patterson,1993,warner books,http://images.amazon.com/images/p/0446364193.0...,"omaha, nebraska, usa",37.394772
4,7,278418,0449217264,0,alaska,james a. michener,1994,fawcett books,http://images.amazon.com/images/p/0449217264.0...,"omaha, nebraska, usa",37.394772


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89451 entries, 0 to 89450
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  89451 non-null  int64  
 1   Uid         89451 non-null  object 
 2   ISBN        89451 non-null  object 
 3   Rating      89451 non-null  int64  
 4   Title       89451 non-null  object 
 5   Author      89451 non-null  object 
 6   Pub_Year    89451 non-null  int64  
 7   Publisher   89451 non-null  object 
 8   Image_Url   89451 non-null  object 
 9   Location    89451 non-null  object 
 10  Age         89451 non-null  float64
dtypes: float64(1), int64(3), object(7)
memory usage: 7.5+ MB


## Creating Memory Based Models

In [ ]:
# drop columns which are not really required
df.drop(['Image_Url', 'Location','Publisher','Unnamed: 0' ],axis = 1, inplace = True)

In [ ]:
df.head()

,Uid,ISBN,Rating,Title,Author,Pub_Year,Age
0,278418,0446520802,0,the notebook,nicholas sparks,1996,37.394772
1,278418,0671537458,0,waiting to exhale,terry mcmillan,1995,37.394772
2,278418,0786817070,0,"artemis fowl (artemis fowl, book 1)",eoin colfer,2002,37.394772
3,278418,0446364193,0,along came a spider (alex cross novels),james patterson,1993,37.394772
4,278418,0449217264,0,alaska,james a. michener,1994,37.394772


In [ ]:
df.shape

(89451, 7)

# **KNN based Recommendation system**


In [ ]:
# Create a pivot table
df_pivot = df.pivot_table(index = 'Title',columns = 'Uid', values = 'Rating',).fillna(0)
df_pivot.head()

Uid,100088,100264,100459,100644,100846,100906,101209,101305,101606,101876,...,9747,97754,98391,9856,98723,98741,98758,98787,98904,99955
Title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to die: a novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: odyssey two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_pivot.shape

(1144, 1148)

In [ ]:
# create a sparse matrix from df_pivot
from scipy.sparse import csr_matrix

df_matrix = csr_matrix(df_pivot.values)

In [ ]:
df_matrix.shape

(1144, 1148)

In [ ]:
# create a Nearest neighbors model with cosine similarity  
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute') 
model_knn.fit(df_matrix) 

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
df_pivot.shape

(1144, 1148)

In [ ]:
#select a random book index and then find its distance and indices
query_index = np.random.choice(df_pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(df_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

78


In [ ]:
# function to find out the top 5 similar books
def top_5_books(query_index):
  distances, indices = model_knn.kneighbors(df_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
  for i in range(0, len(distances.flatten())):
      if i == 0:
          print('Recommendations for {0}:\n'.format(df_pivot.index[query_index]))
      else:
          print('{0}: {1}, with distance of {2}:'.format(i, df_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

In [ ]:

# get a name of book from the user and recommend him 5 books using top_5_books function
book = input('enter the name of book you like: ')
try:
  n = df_pivot.index == book
  query_index = int(np.where(n == True)[0])
  top_5_books(query_index)
except:
  print(f'try for another book, Book not Found')

enter the name of book you like: the notebook
Recommendations for the notebook:

1: a walk to remember, with distance of 0.7326657538383645:
2: nights in rodanthe, with distance of 0.7453577801080011:
3: the rescue, with distance of 0.7670361381874238:
4: message in a bottle, with distance of 0.7812529772709633:
5: toxin, with distance of 0.7818726608333926:


# **Correlation based Recommendation system**

In [ ]:
df_pivot.head()

Uid,100088,100264,100459,100644,100846,100906,101209,101305,101606,101876,...,9747,97754,98391,9856,98723,98741,98758,98787,98904,99955
Title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to die: a novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: odyssey two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Transpose the matrix
ndf_pivot = df_pivot.T

In [ ]:
def corr_recomendation(book):
  cor_vector = ndf_pivot.corrwith( ndf_pivot[book], )
  cor_vector.sort_values(ascending= False, inplace= True)
  print(f"top movies similar to {cor_vector.iloc[:1].index } are")
  print(f'{cor_vector.iloc[2:7] }', end = '\n')

In [ ]:
book = input('enter the name of book you like: ')
try:
  corr_recomendation(book)
except:
  print('your book is not in list, Please Retry....')

enter the name of book you like: the notebook
top movies similar to Index(['the notebook'], dtype='object', name='Title') are
Title
nights in rodanthe                    0.230055
the rescue                            0.201472
toxin                                 0.197444
message in a bottle                   0.176519
the five people you meet in heaven    0.154547
dtype: float64


### Issues
extremely Time consuming memory consuming

# Model Based recommendation engine
using collaborative filtering

---
Latent filter SVD model

In [ ]:
# df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
# rating df, to work on the useful features
rating_df = df[['Uid',	'ISBN',	'Rating',	'Title']].reset_index().drop('index', axis = 1)

In [ ]:
rating_df.head()

,Uid,ISBN,Rating,Title
0,278418,0446520802,0,the notebook
1,278418,0671537458,0,waiting to exhale
2,278418,0786817070,0,"artemis fowl (artemis fowl, book 1)"
3,278418,0446364193,0,along came a spider (alex cross novels)
4,278418,0449217264,0,alaska


In [ ]:
rating_df['Uid'].value_counts()

11676     1340
35859      573
76352      514
16795      474
153662     446
          ... 
11224       30
139387      30
216099      30
32721       30
57725       30
Name: Uid, Length: 1148, dtype: int64

## Stratified Train test Split 

In [ ]:
 # train test split
rating_train_df, rating_test_df = train_test_split(rating_df, 
                                                   test_size = 0.2, 
                                                   random_state = 10,
                                                   stratify= rating_df['Uid'])
print(f"train shape {rating_train_df.shape}")
print(f"test shape {rating_test_df.shape}")

train shape (71560, 4)
test shape (17891, 4)


In [ ]:
# create a pivot of train set
ratings_train_pivot_df = rating_train_df.pivot_table(values = 'Rating', columns = 'ISBN', index = 'Uid').fillna(0)

In [ ]:
# keep a list of user id's for an instace to use them further
user_ids = list(ratings_train_pivot_df.index) 
# keep values of ratings train df as a matrix
ratings_train_values = ratings_train_pivot_df.values

In [ ]:
ratings_train_pivot_df.head(1)

ISBN,0001047973,0006177379,0007110928,0020412401,0027000303,0048231398,0060008024,0060008032,0060008296,0060008776,...,2290303348,3190025355,3257208626,3426616904,3895084573,5551154718,5552027574,8186939008,8401426715,8445072773
Uid,,,,,,,,,,,,,,,,,,,,,
100088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Svd Model to create factor matrix of original matrix

In [ ]:

# The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(ratings_train_values, k = NUMBER_OF_FACTORS_MF)



In [ ]:

# see the shapes of original matrix and all other matrix
print(f' shape of original matrix: {ratings_train_pivot_df.shape}')
print(f' shape of U factor matrix: {U.shape}')
print(f' shape of V-transpose matrix: {Vt.shape}')
sigma = np.diag(sigma) # To Create a diagnol matrix of the sigma
print(f' shape of sigma matrix: {sigma.shape}') 



 shape of original matrix: (1148, 3043)
 shape of U factor matrix: (1148, 15)
 shape of V-transpose matrix: (15, 3043)
 shape of sigma matrix: (15, 15)


## Predict Rating for all users

In [ ]:
# multiply all small matrix to create a predicted final matrix for user ratings 
predicted_rating_for_users = np.dot(np.dot(U,sigma),Vt) 
# predicted rating matrix shape 
predicted_rating_for_users.shape 

(1148, 3043)

In [ ]:
#Converting the reconstructed matrix back to a Pandas dataframe  
user_ids = [str(s) for s in user_ids]
col = [str(c) for c in  ratings_train_pivot_df.columns]
pred_df = pd.DataFrame(predicted_rating_for_users, columns = col, index=user_ids).transpose()  
pred_df.head() 

,100088,100264,100459,100644,100846,100906,101209,101305,101606,101876,...,9747,97754,98391,9856,98723,98741,98758,98787,98904,99955
0001047973,-5.399709e-03,1.468542e-03,1.196391e-02,2.459273e-02,1.792534e-02,2.273521e-02,7.896953e-03,2.273502e-02,6.484844e-03,3.691213e-02,...,-1.165762e-02,3.947408e-03,-4.216465e-01,7.779715e-03,-1.186198e-02,8.830498e-04,-8.318829e-03,2.162063e-02,-2.041958e-03,3.224063e-03
0006177379,-1.171346e-15,-5.445423e-16,-1.536137e-15,7.091368e-16,8.221062e-16,-5.377836e-16,4.795093e-17,4.049230e-17,-8.888184e-16,-4.176434e-16,...,1.077267e-16,-2.579013e-16,6.979866e-15,-6.914199e-16,2.254191e-16,7.827409e-17,-1.549931e-15,-9.191184e-17,-4.033170e-16,-1.949460e-17
0007110928,-1.791098e-02,1.766775e-03,1.988022e-02,1.149981e-01,2.264065e-02,4.819314e-02,4.172850e-03,2.216476e-02,1.355090e-02,5.542668e-02,...,1.946267e-02,4.561425e-02,-2.678880e-01,-4.521556e-03,1.749116e-02,3.059951e-03,-1.704496e-02,1.248535e-01,5.971199e-02,3.209933e-03
0020412401,7.212956e-16,3.445087e-16,1.716422e-15,-8.510355e-16,-6.391092e-16,4.507612e-16,-7.959579e-18,1.109903e-16,1.044435e-15,3.445306e-16,...,-5.274275e-17,2.944119e-16,-6.057754e-15,4.174603e-16,-2.152170e-16,5.717139e-18,1.030705e-15,-4.424387e-17,6.115205e-16,8.055732e-17
0027000303,2.247965e-16,6.962626e-17,9.116683e-17,6.899426e-17,3.330450e-17,-5.385809e-17,3.695490e-17,-3.335207e-16,-3.505481e-16,-1.252316e-16,...,-5.486086e-17,-1.649549e-16,-4.025893e-16,-3.046626e-17,7.940344e-17,-4.989790e-18,2.343566e-16,-2.839541e-16,-1.695526e-16,-3.127632e-17


In [ ]:
# try:
#   books.set_index('ISBN', inplace = True)
# except:
#   pass
# books.reset_index( inplace = True)

In [ ]:
# class for recommendation model 
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, predictions_df, items_df=None):
        self.predictions_df = predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items_by_user(self, user_id, items_to_ignore=[], topn=10, verbose=True):
        # Get and sort the user's predictions
        sorted_user_predictions = self.predictions_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating content that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions.index.isin(items_to_ignore)].sort_values('recStrength', ascending = False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            
            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'index', 
                                                          right_on = 'ISBN')[['recStrength', 'ISBN', 'Book-Title', 
                                                                              'Year-Of-Publication','Publisher',
                                                                              'Book-Author']]

        return recommendations_df

    def recommend_items_by_item(self, book_id,topn=10, verbose=True):
      pdt = self.predictions_df.transpose()
      cosine_similarities = cosine_similarity(np.array(pdt[book_id]).reshape(1,-1),pred_df )
      recommended_df = pd.DataFrame( {'recStrength': cosine_similarities[0] }, index = pred_df.index ).sort_values(by = 'recStrength',ascending = False)
      recommended_df = recommended_df.iloc[:topn,] 
      # print(recommended_df)
      if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            # print(recommended_df)
            # lower case the Book-Title of books DataFrame if not to avoid errors
            self.items_df['Book-Title'] = self.items_df['Book-Title'].str.lower()
            recommended_df = recommended_df.merge(self.items_df, how = 'left', 
                                                          # left_on = 'index', 
                                                          left_index = True,
                                                          right_on = 'ISBN')[['recStrength', 'ISBN', 'Book-Title', 
                                                                              'Year-Of-Publication','Publisher',
                                                                              'Book-Author']]
      return recommended_df

cf_recommender_model = CFRecommender(pred_df, books)




In [ ]:
# This Function is useful only when we need to find 
# function to get all readen books by a user
def get_readen_books(u_id):
  read_notread = ratings_train_pivot_df.loc[u_id ,]
  readen = list(read_notread.iloc[read_notread.values!= 0].index) 
  return readen
get_readen_books

<function __main__.get_readen_books(u_id)>

,recStrength,ISBN,Book-Title,Year-Of-Publication,Publisher,Book-Author
0,2.927338,0440211727,A Time to Kill,1992,Dell,JOHN GRISHAM
1,2.672418,0440214041,The Pelican Brief,1993,Dell,John Grisham
2,2.595368,0345337662,Interview with the Vampire,1993,Ballantine Books,Anne Rice
3,2.241084,0440213525,The Client,1994,Dell Publishing Company,John Grisham
4,2.151656,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,1998,Warner Books,Billie Letts
5,2.059044,0312195516,The Red Tent (Bestselling Backlist),1998,Picador USA,Anita Diamant
6,2.052845,0345313860,"The Vampire Lestat (Vampire Chronicles, Book II)",1986,Ballantine Books,ANNE RICE
7,1.852674,0452282152,Girl with a Pearl Earring,2001,Plume Books,Tracy Chevalier
8,1.765132,0440220602,The Chamber,1995,Dell Publishing Company,John Grisham
9,1.729783,0440226430,Summer Sisters,1999,Dell Publishing Company,Judy Blume


In [ ]:
get_readen_books('100459')[:6]
# get_readen_books('100459')

['0060175400',
 '0060199652',
 '0060392452',
 '0064400557',
 '0142000205',
 '0156005891']

In [ ]:
def get_items_interacted(id, rating_df = rating_df):
  res = rating_df[rating_df['Uid'] == id].sort_values( by = 'Rating', ascending=False)['ISBN']
  return res

In [ ]:
set(read).intersection(get_items_interacted(100459))

set()

# **Evaluation**

### In Recommender Systems, there are a set metrics commonly used for evaluation. We choose to work with **Top-N accuracy metrics**, which evaluates the accuracy of the top recommendations provided to a user, comparing to the items the user has actually interacted in test set.

### This evaluation method works as follows:

* ### For each user
    * ### For each item the user has interacted in test set
        * ### Sample 100 other items the user has never interacted.   
        * ### Ask the recommender model to produce a ranked list of recommended items, from a set composed of one interacted item and the 100 non-interacted items
        * ### Compute the Top-N accuracy metrics for this user and interacted item from the recommendations ranked list
* ### Aggregate the global Top-N accuracy metrics

In [ ]:
#Indexing by personId to speed up the searches during evaluation
rating_indexed_df = rating_df.set_index('Uid')
rating_train_indexed_df = rating_train_df.set_index('Uid')
rating_test_indexed_df = rating_test_df.set_index('Uid')

In [ ]:
df.head(1)

,Uid,ISBN,Rating,Title,Author,Pub_Year,Age
0,278418,0446520802,0,the notebook,nicholas sparks,1996,37.394772


In [ ]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:

    # Function for getting the set of items which a user has not interacted with
    def get_not_interacted_items_sample(self, Uid, sample_size, seed=42):
        interacted_items = get_items_interacted(str(Uid))
        all_items = set(df['ISBN'])
        # print(type(all_items), type(interacted_items))
        non_interacted_items = all_items - set(interacted_items)

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    # Function to verify whether a particular item_id was present in the set of top N recommended items
    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index
    
    # Function to evaluate the performance of model for each user
    def evaluate_model_for_user(self, model, Uid):
        
        # Getting the items in test set
        interacted_values_testset = rating_test_indexed_df.loc[Uid]
        
        if type(interacted_values_testset['ISBN']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['ISBN'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['ISBN'])])
            
        interacted_items_count_testset = len(person_interacted_items_testset) 

        # Getting a ranked recommendation list from the model for a given user
        person_recs_df = model.recommend_items_by_user(Uid, items_to_ignore=get_items_interacted(str(Uid)),topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        
        # For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            
            # Getting a random sample of 100 items the user has not interacted with
            non_interacted_items_sample = self.get_not_interacted_items_sample(Uid, sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, seed=int(item_id)%(2**32))

            # Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            # Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['ISBN'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['ISBN'].values
            
            # Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        # Recall is the rate of the interacted items that are ranked among the Top-N recommended items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    
    # Function to evaluate the performance of model at overall level
    def evaluate_model(self, model):
        
        people_metrics = []
        
        for idx, Uid in enumerate(list(rating_test_indexed_df.index.unique().values)):    
            person_metrics = self.evaluate_model_for_user(model, Uid)  
            person_metrics['_person_id'] = Uid
            people_metrics.append(person_metrics)
            
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics).sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()    

In [ ]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)

print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
1147 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 0.24794589458386898, 'recall@10': 0.3776200324185345}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
120,53,88,268,0.197761,0.328358,11676
126,22,39,115,0.191304,0.339130,35859
75,11,31,103,0.106796,0.300971,76352
103,13,29,95,0.136842,0.305263,16795
91,18,24,89,0.202247,0.269663,153662
62,10,17,84,0.119048,0.202381,102967
9,13,23,77,0.168831,0.298701,230522
18,11,20,76,0.144737,0.263158,78783
265,9,16,75,0.120000,0.213333,55492
12,14,24,75,0.186667,0.320000,185233


Evaluating the Collaborative Filtering model (SVD matrix factorization), we observe that we got **Recall@5 (24%)** and **Recall@10 (38%)** 